In [1]:

!nvidia-smi

Sun Jun 18 08:39:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title Install dependencies
!pip install torch==2.0.0 git+https://github.com/huggingface/diffusers transformers accelerate imageio[ffmpeg]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-gq5vr1_n
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-gq5vr1_n
  Resolved https://github.com/huggingface/diffusers to commit f7cc9adc05c36985c8b3f3e95a54940ddac5b4ca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
#@title Setup pipeline
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video
from IPython.display import HTML
from base64 import b64encode

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

In [4]:
#@title Generate your video
prompt = 'Spiderman chatting with a llama' #@param {type:"string"}
video_duration_seconds = 3 #@param {type:"integer"}
num_frames = video_duration_seconds * 10
video_frames = pipe(prompt, negative_prompt="low quality", num_inference_steps=25, num_frames=num_frames).frames
video_path = export_to_video(video_frames)

  0%|          | 0/25 [00:00<?, ?it/s]

In [5]:
#@title Display the video
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML

def display_video(video):
    fig = plt.figure(figsize=(4.2,4.2))  #Display size specification
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    mov = []
    for i in range(len(video)):  #Append videos one by one to mov
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        mov.append([img])

    #Animation creation
    anime = animation.ArtistAnimation(fig, mov, interval=100, repeat_delay=1000)

    plt.close()
    return anime
video = imageio.mimread(video_path)  #Loading video
HTML(display_video(video).to_html5_video())  #Inline video display in HTML5

In [6]:
pip install git+https://github.com/huggingface/diffusers transformers accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-atz5d_vm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-atz5d_vm
  Resolved https://github.com/huggingface/diffusers to commit f7cc9adc05c36985c8b3f3e95a54940ddac5b4ca
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

# load pipeline
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# optimize for GPU memory
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

In [8]:
!pip install python-telegram-bot==13.7 --force-reinstall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached python_telegram_bot-13.7-py3-none-any.whl (490 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Using cached tornado-6.3.2-cp38-abi3-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (426 kB)
  Using cached APScheduler-3.6.3-py2.py3-none-any.whl (58 kB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
  Using cached setuptools-67.8.0-py3-none-any.whl (1.1 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached tzlocal-5.0.1-py3-none-any.whl (20 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2023.3
    Uninstalling pytz-2023.3:
      Successfully uninstalled pytz-2023.3
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzlocal-5.0.1:
      Successfully uninstalled tzlocal-5.

In [9]:
import telegram
from telegram import Update
from telegram.ext import CommandHandler, MessageHandler, CallbackContext
from telegram.ext import Filters

TOKEN = '5990472072:AAFWCKYDEXodiZSStuHAdPjZMvnNfWjIn7w'

In [12]:
# Define the command handler for the /start command
def start(update: Update, context: CallbackContext):
    update.message.reply_text('Welcome To AI Lawyer! Please send me your question . I am Happy to Assist You.')

# Define the message handler for user's questions
def handle_message(update: Update, context: CallbackContext):
    """Handle user's questions and provide answers."""
    # Get the user's question
    question = update.message.text
    prompt = question
    update.message.reply_text("Please wait for few seconds.")

    # generate

    video_frames = pipe(prompt, num_inference_steps=25, num_frames=200).frames

    # convent to video
    video_path = export_to_video(video_frames)


    # Send the best answer back to the user


    bot.send_video(chat_id=update.message.chat_id, video = open(video_path , 'rb'), supports_streaming = True)


In [ ]:
# Create the Telegram bot and dispatcher
bot = telegram.Bot(token=TOKEN)
updater = telegram.ext.Updater(bot=bot, use_context=True)
dispatcher = updater.dispatcher

# Register the handlers
dispatcher.add_handler(CommandHandler("start", start))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

# Start the bot
updater.start_polling()
updater.idle()

  0%|          | 0/25 [00:00<?, ?it/s]